In [39]:
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine
#from config import username, password
import psycopg2

In [40]:
#read cities data file
wv1 = pd.read_csv("Resources\WV1.csv",header=[0,3])
wv1.head()

,Unnamed: 0_level_0,Population,Surface area,Population density,"Gross national income, Atlas method","Gross national income per capita, Atlas method",Purchasing power parity gross national income,Unnamed: 7_level_0,Gross domestic product,Unnamed: 9_level_0
,Unnamed: 0_level_1,2019,2019,2019,2019,2019,2019,2019,2019,2019
0,Afghanistan,38,652.9,57,20,530,83.3,"2,190",3.9,1.5
1,Albania,2.9,28.8,105,14.9,"5,220",41.3,"14,450",2.2,2.7
2,Algeria,43.1,"2,381.70",18,172.5,"4,010",504.8,"11,720",0.8,-1.1
3,American Samoa,0.1,0.2,277,..,..,..,..,2.2,2.5
4,Andorra,0.1,0.5,164,..,..,..,..,1.8,1.7


In [41]:
wv1.columns=wv1.columns.map("_".join)
wv1.head()

,Unnamed: 0_level_0_Unnamed: 0_level_1,Population_2019,Surface area_2019,Population density_2019,"Gross national income, Atlas method_2019","Gross national income per capita, Atlas method_2019",Purchasing power parity gross national income_2019,Unnamed: 7_level_0_2019,Gross domestic product_2019,Unnamed: 9_level_0_2019
0,Afghanistan,38,652.9,57,20,530,83.3,"2,190",3.9,1.5
1,Albania,2.9,28.8,105,14.9,"5,220",41.3,"14,450",2.2,2.7
2,Algeria,43.1,"2,381.70",18,172.5,"4,010",504.8,"11,720",0.8,-1.1
3,American Samoa,0.1,0.2,277,..,..,..,..,2.2,2.5
4,Andorra,0.1,0.5,164,..,..,..,..,1.8,1.7


In [42]:
#clean header name
wv1.columns = [x.lower().replace(" ","_").replace(",","") for x in wv1.columns]
                    
wv1.head()

,unnamed:_0_level_0_unnamed:_0_level_1,population_2019,surface_area_2019,population_density_2019,gross_national_income_atlas_method_2019,gross_national_income_per_capita_atlas_method_2019,purchasing_power_parity_gross_national_income_2019,unnamed:_7_level_0_2019,gross_domestic_product_2019,unnamed:_9_level_0_2019
0,Afghanistan,38,652.9,57,20,530,83.3,"2,190",3.9,1.5
1,Albania,2.9,28.8,105,14.9,"5,220",41.3,"14,450",2.2,2.7
2,Algeria,43.1,"2,381.70",18,172.5,"4,010",504.8,"11,720",0.8,-1.1
3,American Samoa,0.1,0.2,277,..,..,..,..,2.2,2.5
4,Andorra,0.1,0.5,164,..,..,..,..,1.8,1.7


In [43]:
wv1_df=wv1.rename(columns ={"unnamed:_0_level_0_unnamed:_0_level_1":"country","unnamed:_7_level_0_2019":"per_capita_2019",\
                          "unnamed:_9_level_0_2019":"per_capita_growth_2019"})
wv1_df.head()

,country,population_2019,surface_area_2019,population_density_2019,gross_national_income_atlas_method_2019,gross_national_income_per_capita_atlas_method_2019,purchasing_power_parity_gross_national_income_2019,per_capita_2019,gross_domestic_product_2019,per_capita_growth_2019
0,Afghanistan,38,652.9,57,20,530,83.3,"2,190",3.9,1.5
1,Albania,2.9,28.8,105,14.9,"5,220",41.3,"14,450",2.2,2.7
2,Algeria,43.1,"2,381.70",18,172.5,"4,010",504.8,"11,720",0.8,-1.1
3,American Samoa,0.1,0.2,277,..,..,..,..,2.2,2.5
4,Andorra,0.1,0.5,164,..,..,..,..,1.8,1.7


In [ ]:
wv1_df.dtypes

In [6]:
wv2 = pd.read_csv("Resources\WV2.csv",header=[0,3])
#wv2.head()

In [7]:
wv2.columns=wv2.columns.map("_".join)
#wv2.head()

In [8]:
wv2.columns = [x.lower().replace(" ","_").replace(",","").replace("-","_") for x in wv2.columns]
                    
#wv2.head()

In [ ]:
wv2.columns

In [9]:
wv2_df=wv2.rename(columns ={"unnamed:_0_level_0_unnamed:_0_level_1":"country","unnamed:_10_level_0_2019":"contributing_family_workers_and_own_account_workers_female",\
                           'incidence_of_hiv_ages_15_49_(per_1000_uninfected_population_ages_15_49)_2020':'incidence_of_hiv_ages_15-49_per1000_2020'})
#wv2_df.head()

In [10]:
wv3 = pd.read_csv("Resources\WV3.csv",header=[0,3])
#wv3.head()

In [11]:
wv3.columns=wv3.columns.map("_".join)
#wv3.head()

In [12]:
wv3.columns = [x.lower().replace(" ","_").replace("&","") for x in wv3.columns]
                    
#wv3.head()

In [13]:
wv3_df=wv3.rename(columns ={"unnamed:_0_level_0_unnamed:_0_level_1":"country"})
#wv3_df.head()

In [14]:
wv1_df.drop(wv1.index[227:400],inplace=True)
wv2_df.drop(wv2.index[227:400],inplace=True)
wv3_df.drop(wv3.index[227:400],inplace=True)

In [15]:
wv1_df1=wv1_df.replace(to_replace="..", value="")
wv2_df1=wv2_df.replace(to_replace="..", value="")
wv3_df1=wv3_df.replace(to_replace="..", value="")

In [ ]:
wv1_df1.dtypes

In [ ]:
create table wv1
(
country                                               varchar,
population_2019                                       float,
surface_area_2019                                     float,
population_density_2019                               float,
gross_national_income_atlas_method_2019               float,
gross_national_income_per_capita_atlas_method_2019    float,
purchasing_power_parity_gross_national_income_2019    float,
per_capita_2019                                       float,
gross_domestic_product_2019                           float,
per_capita_growth_2019                                float
);

In [16]:
replacements = {'object': 'Varchar',
    'object': 'float',
  'object': 'float',
  'object': 'float',
  'object': 'int',
  'object': 'float',
  'object': 'float',
  'object': 'float',
  'object': 'float',
  'object': 'float'
}
replacements

{'object': 'float'}

In [17]:
col_str = ", " .join("{} {}".format(n, d) for (n, d) in zip(wv1_df1.columns, wv1_df1.dtypes.replace(replacements)))
col_str 

'country float, population_2019 float, surface_area_2019 float, population_density_2019 float, gross_national_income_atlas_method_2019 float, gross_national_income_per_capita_atlas_method_2019 float, purchasing_power_parity_gross_national_income_2019 float, per_capita_2019 float, gross_domestic_product_2019 float, per_capita_growth_2019 float'

In [18]:
connection = psycopg2.connect(database="world_dev_db", user='postgres', password='bootcamp', host='127.0.0.1', port= '5432')
#
cursor=connection.cursor()
print('opened database successfully')

opened database successfully


In [ ]:
cursor.execute("drop table if exists wy1;")

In [19]:
#create table
cursor.execute("create table wy1\
(country varchar, population_2019 float, surface_area_2019 float, population_density_2019 float, gross_national_income_atlas_method_2019 float, gross_national_income_per_capita_atlas_method_2019 float, purchasing_power_parity_gross_national_income_2019 float, per_capita_2019 float, gross_domestic_product_2019 float, per_capita_growth_2019 float)")

In [21]:
wv1_df1.to_csv('Resources\wv1_final.csv',index=False, encoding = 'utf-8')
wv2_df1.to_csv('Resources\wv2_final.csv',index=False, encoding = 'utf-8')
wv3_df1.to_csv('Resources\wv3_final.csv',index=False, encoding = 'utf-8')

In [25]:
my_file1=open('resources\wv1_final.csv')

In [38]:
SQL_STATEMENT = """
COPY wv1 FROM STDIN WITH
    CSV
    HEADER
    DELIMITER AS ","

cursor.copy_expert(sql=SQL_STATEMENT, file=my_file1)
"""
print('file copied to db')

file copied to db
